<a href="https://colab.research.google.com/github/boodie04/Supervised_ML/blob/main/CNN_DOGS_STANFORD.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Stanford Dogs Classification Using Pretrained Model

 Explanation: We import TensorFlow for deep learning, TensorFlow Datasets to load Stanford Dogs, and matplotlib/numpy for plotting and array manipulation.

In [1]:
import tensorflow as tf
import tensorflow_datasets as tfds
import matplotlib.pyplot as plt
import numpy as np


📥 Step 2: Load the Stanford Dogs Dataset

In [2]:
(train_ds, test_ds), ds_info = tfds.load(
    'stanford_dogs',
    split=['train', 'test'],
    with_info=True,
    as_supervised=True  # returns (image, label) pairs
)


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

KeyboardInterrupt: 

🧹 Step 3: Preprocessing Functions

In [ ]:
IMG_SIZE = 224  # for pretrained models like MobileNetV2


def preprocess(image, label):
    image = tf.image.resize(image, (IMG_SIZE, IMG_SIZE))  # Resize all to 224x224
    image = tf.cast(image, tf.float32) / 255.0             # Normalize to [0, 1]
    return image, label

Resizes the image to 224 × 224 pixels.
This is needed because input images might be different sizes, but the model needs a fixed size to work properly.

tf.cast(image, tf.float32): Converts the image’s pixel values to float numbers (from integers like 0–255 to floats like 0.0–255.0).
/ 255.0: Divides every pixel by 255 so values are in the range [0, 1] instead of [0, 255].
Why?

Neural networks train better and faster when inputs are in a small, consistent range like [0, 1].


🌱 Step 4: Data Augmentation (Only for Augmented Model)

In [ ]:
data_augmentation = tf.keras.Sequential([
    tf.keras.layers.RandomFlip("horizontal"),
    tf.keras.layers.RandomRotation(0.1),
    tf.keras.layers.RandomZoom(0.1),
])


1. tf.keras.layers.RandomFlip("horizontal")

Randomly flips the image left to right (like a mirror).
It helps because in real life, a dog can face either left or right — the model should learn both.
Example:
Original → Dog facing left
Flipped → Dog facing right

2. tf.keras.layers.RandomRotation(0.1)

Rotates the image randomly by up to 10% of 360°, which is ±36 degrees.
This helps the model learn to recognize objects even if they’re tilted a bit.
3. tf.keras.layers.RandomZoom(0.1)

Zooms in or out of the image by up to 10% randomly.
This simulates being closer to or farther from the object.



📦 Step 5: Prepare Datasets (Batching, Shuffling, Caching)

In [ ]:
BATCH_SIZE = 32

train_ds_no_aug = train_ds.map(preprocess).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
train_ds_aug = train_ds.map(
    lambda x, y: (data_augmentation(preprocess(x, y)[0], training=True), y),
    num_parallel_calls=tf.data.AUTOTUNE
).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
#train_ds_aug = train_ds.map(lambda x, y: (data_augmentation(preprocess(x, y)[0]), y)).shuffle(1000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_ds = test_ds.map(preprocess).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


Step	What it does

train_ds.map(preprocess)	Applies your preprocess() function to each (image, label) pair.

.shuffle(1000)	Randomly shuffles the dataset using a buffer size of 1000 (good for training).

.batch(BATCH_SIZE)	Groups images/labels into batches of 32.

.prefetch(tf.data.AUTOTUNE)	Starts loading the next batch while the model is training on the current one, making training faster.

🧠 Step 6: Load a Pretrained Model (e.g., MobileNetV2)

In [ ]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMG_SIZE, IMG_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False  # Freeze the convolutional base




**include_top=False:**

This means you're excluding the final classification layer (the "top" layer) of the model. Normally, MobileNetV2 would end with a fully connected layer that performs the classification. By setting include_top=False, you're only using the convolutional base of the model (which does the feature extraction).


**weights='imagenet':**

This tells Keras to load the pre-trained weights from ImageNet. So, instead of starting the training from scratch, MobileNetV2 will have learned useful features from ImageNet, which can be reused for your task.

**base_model.trainable = False:**

This line is crucial. By setting the trainable property of the base_model to False, you freeze the weights of the convolutional layers. This means that when you train the model:
The convolutional base (which extracts features) will not be updated during backpropagation.
Only the new layers that you add on top of the base model will be trained.

🏗 Step 7: Build the Classifier Model

In [ ]:
def build_model():
    model = tf.keras.Sequential([
        base_model,
        tf.keras.layers.GlobalAveragePooling2D(),
        tf.keras.layers.Dense(120, activation='softmax')  # 120 dog breeds
    ])
    model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics=['accuracy'])
    return model


🏃‍♂️ Step 8: Train Both Models

In [ ]:
model_no_aug = build_model()
model_aug = build_model()

history_no_aug = model_no_aug.fit(train_ds_no_aug, validation_data=test_ds, epochs=10)
history_aug = model_aug.fit(train_ds_aug, validation_data=test_ds, epochs=10)


In [ ]:
model_no_aug = tf.keras.models.load_model('saved_model/model_no_aug')
model_aug = tf.keras.models.load_model('saved_model/model_aug')


In [ ]:
def plot_metrics(history1, history2, metric, title):
    plt.plot(history1.history[metric], label='No Aug')
    plt.plot(history2.history[metric], label='Aug')
    plt.title(title)
    plt.xlabel('Epoch')
    plt.ylabel(metric)
    plt.legend()
    plt.show()

plot_metrics(history_no_aug, history_aug, 'loss', 'Train Loss')
plot_metrics(history_no_aug, history_aug, 'val_loss', 'Validation Loss')
plot_metrics(history_no_aug, history_aug, 'accuracy', 'Train Accuracy')
plot_metrics(history_no_aug, history_aug, 'val_accuracy', 'Validation Accuracy')


In [ ]:
def get_final_metrics(history):
    return (history.history['loss'][-1], history.history['accuracy'][-1],
            history.history['val_loss'][-1], history.history['val_accuracy'][-1])

no_aug_results = get_final_metrics(history_no_aug)
aug_results = get_final_metrics(history_aug)

import pandas as pd
df = pd.DataFrame({
    'Model': ['Pretrained_no_aug', 'Pretrained_aug'],
    'Train Loss': [no_aug_results[0], aug_results[0]],
    'Train Accuracy': [no_aug_results[1], aug_results[1]],
    'Validation Loss': [no_aug_results[2], aug_results[2]],
    'Validation Accuracy': [no_aug_results[3], aug_results[3]],
})
df
